# Root Finding: Bisection Method
---

GENERAL PROBLEM: find the real roots of a given function $f(x)$ in the case when closed-form solutions are not available. That is, find the values of $x$ that satisfy the equation

\begin{align}
  f(x) = 0
\end{align}

where $x$ is a real variable, and $f(x)$ is some non-linear function.

IDEA: start with an interval that is known to bracket a root, divide the interval into two subintervals, performing a simple test to determine which subinterval the root is in, and repeat until the desired precision is obtained.

PRE-REQUISITES:
- [None]

REFERENCES:
- [1] DeVries and Hasbun, *A First Course in Computational Physics, 2nd edition*.
- [2] Burden and Faires, *Numerical Analysis, 7th edition*.
- [3] Ralston and Rabinowitz, *A First Course in Numerical Analysis, 2nd edition*.
- [4] Press et al, *Numerical Recipes: the Art of Scientific Computing, 3rd edition*.

## 1. Summary of the method

Consider a function $f(x)$ with one and only one root "bracketed" in the interval $[a,b]$. Here bracketed specifically means that the function has opposite signs at the end points of the interval. In other words, $f(a)$ and $f(b)$ have opposite signs. 

The idea behind the bisection method is to divide the bracketing interval in half, determine which subinterval still brackets the root, and repeat. The process is continued until the root is located to within some given tolerance, or some maximum allowed number of iterations is reached.

The midpoint of the $i$th iteration is found using 

\begin{align}
  p_{i} = \frac{a_{i} + b_{i}}{2} 
  = a_{i} + \frac{b_{i} - a_{i}}{2}
\end{align}

(PROGRAMMER'S NOTE: the second form is prefered, since it is less susceptible to roundoff error. When $b_{i}-a_{i}$ becomes close to machine precision, $(a_{i}+b_{i})/2$ may return a value outside of the interval $[a_{i},b_{i}]$. The second expression for the midpoint does not have this defect.) 

To determine which half interval the root falls in, define the function

\begin{align}
  \phi(x_{1},x_{2}) \equiv \mathrm{sgn}(f(x_{1}))\mathrm{sgn}(f(x_{2}))
\end{align}

At each iteration, apply this to the left subinterval $\phi(a_{i-1},p_{i-1})=\mathrm{sgn}(f(a_{i-1}))\mathrm{sgn}(f(p_{i-1}))$. There are three cases to consider:

- *case i.* $\phi(a_{i-1},p_{i-1}) < 0$. This means that $f(a_{i-1})$ and $f(p_{i-1})$ have opposite signs, which implies that the root lies in the *left* subinterval $[a_{i-1},p_{i-1}]$. Therefore we choose our new interval to be the left subinterval, setting $[a_{i},b_{i}]=[a_{i-1},p_{i-1}]$.


- *case ii.* $\phi(a_{i-1},p_{i-1}) > 0$. This means that $f(a_{i-1})$ and $f(p_{i-1})$ have the same sign, which implies that the root is not in the left subinterval, and therefore must lie in the *right* subinterval $[p_{i-1},b_{i-1}]$. Therefore we choose our new interval to be the right subinterval, setting $[a_{i},b_{i}]=[p_{i-1},b_{i-1}]$.


- *case iii.* $\phi(a_{i-1},p_{i-1}) = 0$. This means that the midpoint coincides with the location of the root to within machine precision, and so we're done.

(PROGRAMMER'S NOTE: the product $f(a_{i-1})f(p_{i-1})$ could also be used as a test, but it is more more susceptible to overflow/underflow if $f(x)$ becomes too large at either end of the interval.)

To calculate the relative uncertainty in the location of the root at the end of an iteration, take the best estimate of the root, $\bar{x}$, to be the latest midpoint, and take the absolute uncertainty, $\delta{x}$, to be the width of the latest interval. The relative error is then calculated as

\begin{align}
  \mathsf{REL} = \left|\frac{\delta{x}}{\bar{x}}\right| 
  = \left|\frac{\text{(width of current interval)}}{\text{(midpoint of current interval)}}\right|
  = \left|\frac{b_{i} - a_{i}}{p_{i}}\right|
\end{align}

We should also calculate the error of any proposed root candidate  

\begin{align}
  \mathsf{ABS} = \left|f(p_{i})\right|
\end{align}

The process of searching for a root should continue until both of these errors are less than some specified tolerance (or the maximum allowed number of iterations is reached).

We have assumed that one and only one root is bracketed by the initial interval. Under that condition, the root is always bracketed between two end points, and the interval is ever shrinking. So given enough iterations, the method is guaranteed to converge. The root will eventually be located. Of course, if more than one root lies within the initial interval, the method will find one of them, but there is no guarantee which one.

(PROGRAMMER'S NOTE: the assumption that a root is bracketed by the initial interval is crucial. If there is a root in the interval, but it is not bracketed (say if the root is also a minimum or maximum of the function), our sign test will in general fail. Our test assumes that when the sign of the function at two points is the same, there is no root lying between them. A root that is also a minimum or maximum presents a counter-example to this logic. Therefore the bisection method is not equipped to handle this case. As a result, our algorithm should check whether a root is bracketed by the initial interval before applying the method.)

## 2. Algorithm

**INPUT**
- an interval $[a,b]$ where a root of the function in question is known to exist.
- TOL, the relative error tolerance that the answer is required to have.
- $i_\mathrm{max}$, maximum number of iterations allowed.

**Validate initial interval**
- if $\phi(a,b) \geq 0$, complain and quit. Otherwise continue.

**Initialize loop**
- set $i = 0$
- set $a_{0}=a$ and $b_{0}=b$
- set $p_{0} = a_{0} + (b_{0}-a_{0})/2$

**Loop** while $i \leq i_\mathrm{max}$

- i = i + 1


- determine which subinterval the root lies in, and update end points
  - calculate $\phi(a_{i-1},p_{i-1}) = \mathrm{sgn}(f(a_{i-1}))\mathrm{sgn}(f(p_{i-1}))$.
  - if $\phi(a_{i-1},p_{i-1}) < 0$, set $a_{i}=a_{i-1}$ and $b_{i}=p_{i-1}$
  - if $\phi(a_{i-1},p_{i-1}) > 0$, set $a_{i}=p_{i-1}$ and $b_{i}=b_{i-1}$
  - if $\phi(a_{i-1},p_{i-1}) = 0$, stop


- calculate the new midpoint using:  $p_{i} = a_{i} + (b_{i} - a_{i})/2$


- calculate the relative uncertainty using:  REL $= \left|(b_{i} - a_{i})/p_{i}\right|$


- calculate ABS $= |f(p_{i})|$


- if (REL $\leq$ TOL) and (ABS $\leq$ TOL), stop. Otherwise, continue.


- if $i > i_\mathrm{max}$, stop. Otherwise, go back to the top of the loop.

## 3. CODE

In [182]:
%%writefile bisection.py

import numpy as np
import sys

# bisection method
def bisect(f, a, b, TOL, imax):
    """
    searches for roots of f(x) using the bisection method
    
    INPUT:
    f = function whose roots are being sought 
    a = initial left bracket
    b = initial right bracket
    TOL = allowed tolerance
    imax = maximum number of iterations
    
    OUTPUT:
    location of the root to within the allowed tolerance, or failure message
    """
    
    # test initial bracket 
    if np.sign(f(a))*np.sign(f(b)) > 0:
        # if sgn > 0, problem may not be well-defined
        print('ERROR: function has the same sign at both ends of the interval.')
        print('Bisection method is not well-suited to this problem, with the given input.')
        print('Try a different initial interval, or choose a different method. Stopping.')
        return

    # initialize iteration
    xLeft = a   # set left bound
    xRight = b  # set right bound
    xMid = xLeft + (xRight - xLeft)/2. # calculate midpoint
    
    # iterate search using bisection method
    i = 0  # reset iteration number
    while i <= imax:
    
        # increment iteration number
        i = i + 1

        # determine which subinterval the root lies in, and update end points      
        sgn = np.sign(f(xLeft))*np.sign(f(xMid))
        if sgn <= 0:
            # if sgn < 0, root is in left subinterval,
            # make midpt the new right end point
            print('iteration',i,': root is located in left subinterval, between',xLeft,'and',xMid)
            xRight = xMid
        elif sgn > 0:
            # if sgn > 0, root is in right subinterval,
            # make midpt the new left end point
            print('iteration',i,': root is located in right subinterval, between',xMid,'and',xRight)
            xLeft = xMid 
        elif sgn == 0:
            # if sgn = 0, root is located at the midpoint
            print('SUCCESS! Root has been located to within machine precision after',i,'iterations.')
            print('Root is located at',xMid)
            return

        # calculate new midpoint
        xMid = xLeft + (xRight - xLeft)/2.
    
        # calculate errors
        xErr = np.abs((xRight - xLeft)/xMid)
        fErr = f(xMid)

        # check if errors are within the allowed tolerance
        if (xErr <= TOL and fErr <= TOL):
            best = xMid #best estimate
            delta = xRight-xLeft #uncertainty
            print('SUCCESS! Root has been located to within the specified tolerance after',i,'iterations.')
            print('Root is located at',best,'+/-',delta)
            return

        # print message if max iteration has been reached
        if i == imax:
            print('FAIL! Max number of iterations has been reached. Stopping.')
            return

Writing bisection.py


In [183]:
%run bisection.py

## 4. Example

In [8]:
# define function whose roots we are searching for
def myfunc(x):
    return np.cos(x) - x

In [9]:
bisect(myfunc, 0., 1., 1e-9, 100)

iteration 1 : root is located in right subinterval, between 0.5 and 1.0
iteration 2 : root is located in left subinterval, between 0.5 and 0.75
iteration 3 : root is located in right subinterval, between 0.625 and 0.75
iteration 4 : root is located in right subinterval, between 0.6875 and 0.75
iteration 5 : root is located in right subinterval, between 0.71875 and 0.75
iteration 6 : root is located in right subinterval, between 0.734375 and 0.75
iteration 7 : root is located in left subinterval, between 0.734375 and 0.7421875
iteration 8 : root is located in right subinterval, between 0.73828125 and 0.7421875
iteration 9 : root is located in left subinterval, between 0.73828125 and 0.740234375
iteration 10 : root is located in left subinterval, between 0.73828125 and 0.7392578125
iteration 11 : root is located in right subinterval, between 0.73876953125 and 0.7392578125
iteration 12 : root is located in right subinterval, between 0.739013671875 and 0.7392578125
iteration 13 : root is l

## 5. Another example

In [10]:
# define function whose roots we are searching for
def myfunc2(x):
    #return (x-2)**2
    return np.cos(x) - x*np.sin(x)

In [11]:
bisect(myfunc2, 0, 3, 1e-7, 100)

iteration 1 : root is located in left subinterval, between 0 and 1.5
iteration 2 : root is located in right subinterval, between 0.75 and 1.5
iteration 3 : root is located in left subinterval, between 0.75 and 1.125
iteration 4 : root is located in left subinterval, between 0.75 and 0.9375
iteration 5 : root is located in right subinterval, between 0.84375 and 0.9375
iteration 6 : root is located in left subinterval, between 0.84375 and 0.890625
iteration 7 : root is located in left subinterval, between 0.84375 and 0.8671875
iteration 8 : root is located in right subinterval, between 0.85546875 and 0.8671875
iteration 9 : root is located in left subinterval, between 0.85546875 and 0.861328125
iteration 10 : root is located in right subinterval, between 0.8583984375 and 0.861328125
iteration 11 : root is located in right subinterval, between 0.85986328125 and 0.861328125
iteration 12 : root is located in left subinterval, between 0.85986328125 and 0.860595703125
iteration 13 : root is l

## 6. Bracket hunting add-on

In the spirit of defensive programming, we might employ a slightly more sophisticated check on whether a root of a function is bracketed by the initial interval. We might be blindly handed a function that has more than one root in the initial interval. Or more likely, we might mistakenly choose an initial interval that is either too big (so that it includes multiple roots) or too small (so that it contains no roots). Our algorithm above checks whether the initial interval brackets a root, but it gives no insight in the case of a failure.

A simple way to extend our code is to perform an initial search over the given interval for sign changes, which would suggest the presence of bracketed roots. We can make this probe as fancy as we like. Here we just take a brute force approach: divide the interval into $N$ evenly spaced subintervals, and then test each subinterval for a sign change. The routine returns a list of potential bracketed roots. 

In [184]:
%%writefile getbrackets.py

import numpy as np

def get_brackets(f, a, b, n):
    """
    scans an interval for bracketed roots by performing a simple divide and conquer strategy
    
    INPUT:
    f = function to scan for bracketed roots
    a = left end point of search interval
    b = right end point of search interval
    n = number of subdivisions to consider
    
    OUTPUT:
    list of potential bracketed roots
    """
    
    # create grid
    x = np.linspace(a,b,n)
    
    # initialize output grid, adding dummy row (to be removed later)
    keep = np.array([[a,b]]) 
    
    # test for bracketed roots (sign changes)
    for i in range(0,n-1):
        phi = np.sign(f(x[i]))*np.sign(f(x[i+1]))
        if phi < 0:
            print('sign change detected between',x[i],'and',x[i+1])
            keep = np.concatenate((keep, [[x[i],x[i+1]]]))
    
    # remove first row (dummy row)
    keep = np.delete(keep, 0, axis=0)

    # check whether bracketed roots have been detected
    if keep.size == 0:
        print('WARNING: no sign changes have been detected between',a,'and',b)
    
    return keep

Writing getbrackets.py


In [185]:
%run getbrackets.py

In [186]:
roots = get_brackets(myfunc2, -5, 3, 100)

sign change detected between -3.46464646465 and -3.38383838384
sign change detected between -0.878787878788 and -0.79797979798
sign change detected between 0.818181818182 and 0.89898989899


In [180]:
nr = 1
bisect(myfunc2, roots[nr][0], roots[nr][1], 1e-9, 100)

iteration 1 : root is located in left subinterval, between -0.878787878788 and -0.838383838384
iteration 2 : root is located in left subinterval, between -0.878787878788 and -0.858585858586
iteration 3 : root is located in right subinterval, between -0.868686868687 and -0.858585858586
iteration 4 : root is located in right subinterval, between -0.863636363636 and -0.858585858586
iteration 5 : root is located in right subinterval, between -0.861111111111 and -0.858585858586
iteration 6 : root is located in left subinterval, between -0.861111111111 and -0.859848484848
iteration 7 : root is located in right subinterval, between -0.86047979798 and -0.859848484848
iteration 8 : root is located in left subinterval, between -0.86047979798 and -0.860164141414
iteration 9 : root is located in left subinterval, between -0.86047979798 and -0.860321969697
iteration 10 : root is located in right subinterval, between -0.860400883838 and -0.860321969697
iteration 11 : root is located in right subinte

In [181]:
bisect(myfunc2, 0, 1, 1e-9, 100)

iteration 1 : root is located in right subinterval, between 0.5 and 1
iteration 2 : root is located in right subinterval, between 0.75 and 1
iteration 3 : root is located in left subinterval, between 0.75 and 0.875
iteration 4 : root is located in right subinterval, between 0.8125 and 0.875
iteration 5 : root is located in right subinterval, between 0.84375 and 0.875
iteration 6 : root is located in right subinterval, between 0.859375 and 0.875
iteration 7 : root is located in left subinterval, between 0.859375 and 0.8671875
iteration 8 : root is located in left subinterval, between 0.859375 and 0.86328125
iteration 9 : root is located in left subinterval, between 0.859375 and 0.861328125
iteration 10 : root is located in left subinterval, between 0.859375 and 0.8603515625
iteration 11 : root is located in right subinterval, between 0.85986328125 and 0.8603515625
iteration 12 : root is located in right subinterval, between 0.860107421875 and 0.8603515625
iteration 13 : root is located 